<a href="https://colab.research.google.com/github/csgm2328/Emily/blob/master/Max_Mcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import pandas as pd
from pandas import json_normalize
import numpy as np
import plotly as py
import plotly.graph_objs as go
import requests
from keras import metrics
from keras.layers import *
from keras.models import *
from keras.utils.np_utils import to_categorical
from keras.callbacks import *
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
plt.style.use('bmh')
import os
from keras.models import load_model

In [ ]:
start_date = datetime(2017,5,1)
start = int(start_date.timestamp()) 
ret = requests.get('https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start={}&end=9999999999&period=86400'.format(start))
print(ret)

In [ ]:
js = ret.json()
print(js[-1])
df = pd.DataFrame(js)

In [ ]:
scaler = MinMaxScaler()
real_price = df['close'].tolist()
price = scaler.fit_transform(df[['close']]).reshape(-1).tolist()  

print('오늘은',datetime.today(), real_price[-1], len(real_price), '개') # GMT +0 기준

In [ ]:
def make_DataSet(w):
  y = []  

  for i in range(len(price) - w):   
    x.append([price[i+j] for j in range(w)])
    y.append(price[w + i])
  
  x = np.asarray(x)
  y = np.asarray(y)

  train_test_split = int(len(x)*0.7) 

  x_train = x[:train_test_split, :]
  y_train = y[:train_test_split]

  x_test = x[train_test_split:, :]
  y_test = y[train_test_split:]

  xtrain = x_train.reshape((x_train.shape[0], x_train.shape[1], 1)) 
  xtest =  np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  

  return xtrain, xtest, y_train, y_test

In [ ]:
def make_Prediction(window_size, n_batch):
  x = []  
  for i in range(len(price) - window_size): 
    x.append([price[i+j] for j in range(window_size)])
  x = np.asarray(x)
  train_test_split = int(len(x)*0.7)

  x_test = x[train_test_split:, :]
  xtest =  np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

  testing = model.predict(xtest,batch_size=n_batch)
  model.reset_states()
  real_testing = scaler.inverse_transform(testing)
  return testing, real_testing

In [ ]:
#F1 score는 지나치게 긍정적으로 평가되는 경향이 있다. 분류문제인 양수인지 음수인지를 판별할 때는 MCC를 사용한다
#MCC는 -1 ~ +1 사이 값을 나타내며 0은 임의 예측과 같은성능
#방향 정확성 평가, MCC는 상승하락에 대해 균형있게 예측했는지 평가
#Matthews correlation coefficient, confusion Matrix 
def MCC(actual, target, t): 
  TP, TN, FP, FN = 0,0,0,0
  ACC_List = [] 
  MCC_List = []

  for i in range(len(target)):
    if actual[i+t] - actual[i+t-1] > 0: #상승
      if target[i] - actual[i+t-1] > 0: #상승예측
        TP += 1
      else:
        FN += 1
    elif actual[i+t] - actual[i+t-1] < 0: # 하락
      if target[i] - actual[i+t-1] < 0:
        TN += 1
      else:
        FP += 1
    else: #그대로
      if target[i] == actual[i+t-1]:
        TP += 1
      elif target[i] - actual[i+t-1] > 0:
        FP += 1
      else:
        FN += 1
    N = TP + TN + FP + FN
    S = (TP + FN) / N
    P = (TP + FP) / N
    ACC = (TP + TN) / N
    if P == 0 or ACC == 0:
      MCC = np.nan
    else:
      MCC = (TP / N - S * P) / np.sqrt(P*S*(1-S)*(1-P))
    ACC_List.append(ACC)
    MCC_List.append(MCC)

  N = TP + TN + FP + FN
  S = (TP + FN) / N
  P = (TP + FP) / N
  ACC = (TP + TN) / N
  MCC = (TP / N - S * P) / np.sqrt(P*S*(1-S)*(1-P))
  #MCC = (TP * TN - FP * FN) / np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
  #두개가 0이되면 측정안됨
  print(TP,FN,TN,FP)
  print('방향 정확도: %.3f%%\nMCC: %.2f' %(ACC*100,MCC))
  return ACC, MCC, ACC_List, MCC_List

In [ ]:
#keras 라이브러리 내에서 mape 평가는 값이 너무 큼 mae,mse 전부다
def MAPE(actual, target, t): #Mean absolute percentage error
  scores = []
  for i in range(len(target)):
    scores.append(np.abs((actual[i+t] - target[i]) / actual[i+t]))
  print('값 오차율: %.2f%%\n' %(np.average(scores)*100))
  return scores

In [ ]:
#모델마다 돌면서 max mcc, acc list 만들기
def find_max(verif_date, temp, sec):
  max_list = []
  sec_max_pt = np.zeros((len(temp),1))
  max_pt = np.zeros((len(temp),1))
  date = 0 #구간표현용

  for i in range(verif_date): #  검증 날짜수
    max = -9999
    for j in range(len(temp)): #  모델 수   
      if  max < temp[j][i]:
        max = temp[j][i]
      if j == len(temp)-1: # 모델 다 돈 후에 max 값과 같은 값을 가진 모델들에게 sec_max_pt + 1
        for k in range(len(temp)):
          if max == temp[k][i]:
            sec_max_pt[k]+=1

    if max == -9999:
      max = np.nan

    max_list.append(max) #그 날의 가장 우수한 metric값들 저장

    if sec != 0: #구간 평가일때
      if (i+1) % sec == 0: #구간마다 평가
        print('\n<<<<<[%d ~ %d]일 구간 우수모델>>>>>' %(verif_date-date, verif_date-(i+1)))
        date = i+1
        max_rank = sorted(sec_max_pt,reverse=True)
        rank = 5 # 몇위까지?
        for y in range(rank):
          for z in range(len(sec_max_pt)):
            if max_rank[y-1] != max_rank[y] and max_rank[y] == sec_max_pt[z]:
              #z번째의 모델 이름
              print('%d위: [%d점] %s' %(y+1, max_rank[y], model_list[z]))

        max_pt += sec_max_pt
        sec_max_pt = np.zeros((len(temp),1))

  max_pt += sec_max_pt
  print('\n<<<<<%d일 누적 우수모델>>>>>' %verif_date)
  max_rank = sorted(max_pt,reverse=True) 
  rank = 10 
  for y in range(rank):
    for z in range(len(max_pt)):
      if max_rank[y-1] != max_rank[y] and max_rank[y] == max_pt[z]:
        print('%d위: [%d점] %s' %(y+1, max_rank[y], model_list[z]))
        
  return max_list,max_pt

In [ ]:
#구간별 평가
#mcc가 초기화될때마다 TP가 0일때 평가가 안되는 문제는 감수해야함
def sec_metrics(sec, verif_date):
  acc, mcc = [],[]
  ACC_List, MCC_List = [],[]
  MAPE_List = []

  for i in range(-verif_date,0,sec): #검증하는 기간을 구간별로 나눠서 반복
    if i+sec == 0:
      sec_acc, sec_mcc, sec_ACC_List, sec_MCC_List = MCC(real_price[i:],real_testing[i:],0)
      sec_MAPE_List = MAPE(real_price[i:], real_testing[i:], 0)
    else:
      sec_acc, sec_mcc, sec_ACC_List, sec_MCC_List = MCC(real_price[i:i+sec],real_testing[i:i+sec],0)
      sec_MAPE_List= MAPE(real_price[i:i+sec], real_testing[i:i+sec], 0)

    acc.append(sec_acc)
    mcc.append(sec_mcc)
    MAPE_List.append(sec_MAPE_List)  
    ACC_List.append(sec_ACC_List)
    MCC_List.append(sec_MCC_List)

  #sec_MCCs = sum(sec_MCCs,[]) #sum(iterable, start) np.flatten() 보다 1000배 느림 %timeit으로 검사
  #1차원으로
  acc = np.asarray(acc).flatten()
  mcc = np.asarray(mcc).flatten()
  MAPE_List = np.asarray(MAPE_List).flatten()
  ACC_List = np.asarray(ACC_List).flatten()
  MCC_List = np.asarray(MCC_List).flatten()

  return acc, mcc, MAPE_List, ACC_List, MCC_List
  

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/LSTM_models/

In [ ]:
#모델 저장된 폴더에서 모델들 불러오기
model_list = !ls
#model_list = model_list[:10]
len(model_list)

In [ ]:
#따옴표 없애기
for i in range(len(model_list)):
  s = model_list[i]
  s = s[1:]
  s = s[:-1]
  model_list[i] = s

In [ ]:
#epoch 종류 list
ep_list = []
for i in range(len(model_list)):
  ep = model_list[i].split(' ')[7][:-2]
  if ep not in ep_list:
  #if (np.any(ep == ep_list)):
    ep_list.append(ep)
print(ep_list)

#Argument별 평가

In [ ]:
#Argument별 평가
%%time
ep_list = []
#epoch 종류별로 window_size에 따른 mae값 비교
for i in range(len(ep_list)):
  train_maes, test_maes = [],[]
  xaxes = []
  for j in range(len(model_list)):
    if ep_list[i] == model_list[j].split(' ')[7][:-2]: #같은 종류의 ep찾으면
      #evaluate
      n_batch = int(model_list[j].split(' ')[6][:-3])
      window_size = int(model_list[j].split(' ')[5][:-3])

      xtrain, xtest, y_train, y_test = make_DataSet(window_size)
      model = load_model(model_list[j])
      trainScore = model.evaluate(xtrain, y_train, batch_size = n_batch, verbose=0)
      testScore = model.evaluate(xtest, y_test, batch_size = n_batch, verbose=0)

      print(model_list[j])
      print('mae: (%.5f, %.5f)\n' %(trainScore[1],testScore[1]))
      xaxes.append(window_size)
      train_maes.append(trainScore[1])
      test_maes.append(testScore[1])
    
  #graph
  #min 값은?
  print(xaxes)
  
  print('train MIN : %f' %(np.min(train_maes)))
  print('test MIN : %f' %(np.min(test_maes)))

  train_ax = plt.subplot()
  test_ax = train_ax.twinx()

  train_ax.plot(xaxes, train_maes, 'yo', label='train_mae')
  test_ax.plot(xaxes, test_maes, 'bo', label='test_mae')

  train_ax.set_xlabel(ep_list[i]+'ep: window_size')
  #train_ax.set_ylabel('train_mae')
  #test_ax.set_ylabel('test_mae')

  train_ax.legend(loc='best')
  test_ax.legend(loc='best')
   
  plt.show()

  
  

#학습한 모델들 MCC 평가

In [ ]:
#ACC, MCC 평가
%%time
MCCs, ACCs = [],[]
ACC_Lists, MCC_Lists = [],[]
ACC_data, MCC_data = [],[]
MAPE_Lists = []
verif_date = 270
sec = 0  #구간별 

for i in range(len(model_list)):
  model_name = model_list[i]
  model = load_model(model_name)
  window_size = int(model_name.split(' ')[5][:-3])
  n_batch = int(model_name.split(' ')[6][:-3])
  n_epoch = int(model_name.split(' ')[7][:-2])
  name = model_name.split(' ')[1] + ' ~ ' + model_name.split(' ')[3]+ ' ' + str(window_size) + 'win ' + str(n_epoch) + 'ep'


  xtrain, xtest, y_train, y_test = make_DataSet(window_size)
  testing = model.predict(xtest,batch_size=n_batch)
  model.reset_states()
  real_testing = scaler.inverse_transform(testing)
  #testing, real_testing = make_Prediction(window_size, n_batch)
  
  #출력
  print(name)
  if sec != 0: #구간 평가일때
    acc, mcc, MAPE_List, ACC_List, MCC_List = sec_metrics(sec, verif_date)

  else: #누적 평가일때
    acc, mcc, ACC_List,MCC_List = MCC(real_price[-verif_date:], real_testing[-verif_date:], 0)
    MAPE_List = MAPE(real_price[-verif_date:], real_testing[-verif_date:], 0)

  ACCs.append(acc)
  MCCs.append(mcc)
  MAPE_Lists.append(MAPE_List)
  ACC_Lists.append(ACC_List) 
  MCC_Lists.append(MCC_List)

  #make go.scatter data
  x1 = np.arange(0,verif_date,1)
  if acc > 0.5: #50% 이상만 그리기위해
    go_acc = go.Scatter(x=x1, y = ACC_List, line=dict(color='black'), name = name+'[acc]', yaxis='y')
    ACC_data.append(go_acc)
  if mcc > 0.0: 
    go_mcc = go.Scatter(x=x1, y = MCC_List, mode='lines+markers',line=dict(color='blue'), name = name+'[MCC]', yaxis='y2')
    MCC_data.append(go_mcc)

  #go_mape = go.Scatter(x=x1, y = MAPE_List, line=dict(color='yellow'), name = name+'[MAPE]', yaxis='y3')
  #MCC_data.append(go_mcc)
  
#Max찾기
#평균 Max찾기
rank = 10
print('\n<<<<<%d일 누적평균 정확성 우수모델>>>>>' %verif_date)
max_ACCs = sorted(ACCs,reverse=True)
for i in range(rank):
  for j in range(len(ACCs)):
    if max_ACCs[i-1] != max_ACCs[i] and max_ACCs[i] == ACCs[j]:
      print('%d위는 ACC:%.3f%% 의 %s' % (i+1, max_ACCs[i]*100, model_list[j])) # 정확도

print('\n\n<<<<<%d일 누적평균 MCC 우수모델>>>>>' %verif_date)
max_MCCs = sorted(MCCs,reverse=True)
for i in range(rank):
  for j in range(len(MCCs)):
    if max_MCCs[i-1] != max_MCCs[i] and max_MCCs[i] == MCCs[j]:
      print('%d위는 MCC:%.2f 의 %s' % (i+1, max_MCCs[i], model_list[j])) # 정확도
            
acc_max_list,acc_max_pt = find_max(verif_date, ACC_Lists, sec)
mcc_max_list,mcc_max_pt = find_max(verif_date, MCC_Lists, sec)
#mape_max_list = find_max(verif_date, MAPE_Lists)

go_am = go.Scatter(x=x1, y = acc_max_list, line=dict(color='red',width=6),name = 'Max Accuracies', yaxis='y')
go_mm = go.Scatter(x=x1, y = mcc_max_list, mode='lines+markers',line=dict(color='lime',width=6), name = 'Max MCCs', yaxis='y2')
#go_mp = go.Scatter(x=x1, y = mcc_max_list,line=dict(color='magenta',width=6), name = 'Max MAPEs', yaxis='y3')

In [ ]:
cd /content/

In [ ]:
#합쳐서 acc, mcc
Title = '정확도, MCC의 MAX 찾기' + ' 최근' + str(verif_date) + '일 '+ '[' + str(datetime.date(datetime.today())) + ']'
fig = go.Figure()
fig.add_traces(go_am)
fig.add_traces(go_mm)
fig.add_traces(ACC_data)
fig.add_traces(MCC_data)

layout = go.Layout(title=Title,
                   yaxis=dict(
                       title='정확도',zeroline=True, showline=True),
                   yaxis2=dict(
                       title = 'MCC', overlaying='y', side='right', position=1)
)
fig.update_layout(layout)
py.offline.plot(fig)


os.rename('temp-plot.html', '합친 ' + Title + '.html')

In [ ]:
#모델별로 acc & mcc 
from plotly.subplots import make_subplots
rows = 4
cols = 5
fig = make_subplots(rows=rows, cols=cols)
k = 0

for i in range(rows):
  for j in range(cols):
    fig.add_trace(ACC_data[k],row=i+1, col=j+1)
    fig.add_trace(MCC_data[k],row=i+1, col=j+1 )
    k+=1

fig.update_layout(go.Layout(title=Title))
py.offline.plot(fig)

os.rename('temp-plot.html', '모델별 ' + Title + '.html')